# Создание столбцов на ходу
- Лена - разработчик
- Ваня - дизайнер
- Настя - редактор

In [1]:
import pandas as pd

In [16]:
def tickets(row):
#     print(row)
    info = row['dicts']
    # print(info)
    
    if 'Лена' in info:
        row['разработка'] = info['Лена']
        
    if 'Настя' in info:
        row['редактор'] = info['Настя']
    
    return row

In [17]:
df.head().apply(tickets, axis=1)

,dicts,разработка,редактор
0,"{'Лена': 1, 'Ваня': 2}",1.0,NaN
1,"{'Ваня': 3, 'Леша': 5}",NaN,NaN
2,{'Настя': 3},NaN,3.0


In [2]:
df = pd.DataFrame({'dicts': [{'Лена': 1, 'Ваня': 2}, {'Ваня': 3, 'Леша': 5}, {'Настя': 3}]})
df

,dicts
0,"{'Лена': 1, 'Ваня': 2}"
1,"{'Ваня': 3, 'Леша': 5}"
2,{'Настя': 3}


# Группировки и оконные функции

In [2]:
df = pd.DataFrame(
    {
        'region': ['СФО', 'СФО', 'ДВФО', 'СФО', 'ДВФО', 'ДВФО', 'ДВФО', 'ДВФО', 'СФО', None],
        'city': ['Новосибирск', 'Томск', 'Владивосток', 'Новосибирск', 'Владивосток', 'Владивосток', 'Владивосток', 'Владивосток', 'Томск', 'Владивосток'],
        'sales': [140, 135, 290, 95, 100, 50, 40, 120, 150, 400],
    }
)
df

,region,city,sales
0,СФО,Новосибирск,140
1,СФО,Томск,135
2,ДВФО,Владивосток,290
3,СФО,Новосибирск,95
4,ДВФО,Владивосток,100
5,ДВФО,Владивосток,50
6,ДВФО,Владивосток,40
7,ДВФО,Владивосток,120
8,СФО,Томск,150
9,None,Владивосток,400


In [3]:
# что возвращает метод count?
# count возвращает количество НЕПУСТЫХ строк (!!!)
df.groupby('city').count()

,region,sales
city,,
Владивосток,5,6
Новосибирск,2,2
Томск,2,2


In [4]:
len(df)

10

In [21]:
# почему сумма по столбцу не равна 10?
# группировка учитывает ТОЛЬКО НЕПУСТЫЕ ключи
df.groupby('region').count()

,city,sales
region,,
ДВФО,5,5
СФО,4,4


In [22]:
# подсчет уникальных значений
df.groupby('region').nunique()

,city,sales
region,,
ДВФО,1,5
СФО,2,4


Что такое группировка?

In [23]:
df

,region,city,sales
0,СФО,Новосибирск,140
1,СФО,Томск,135
2,ДВФО,Владивосток,290
3,СФО,Новосибирск,95
4,ДВФО,Владивосток,100
5,ДВФО,Владивосток,50
6,ДВФО,Владивосток,40
7,ДВФО,Владивосток,120
8,СФО,Томск,150
9,None,Владивосток,400


In [27]:
df.groupby('city').sum()

,sales
city,
Владивосток,1000
Новосибирск,235
Томск,285


In [24]:
# шаг 1
df['city'].unique()

array(['Новосибирск', 'Томск', 'Владивосток'], dtype=object)

In [25]:
# шаг 2 - берем строчки для одного города
df_vladik = df[df['city'] == 'Владивосток']
df_vladik

,region,city,sales
2,ДВФО,Владивосток,290
4,ДВФО,Владивосток,100
5,ДВФО,Владивосток,50
6,ДВФО,Владивосток,40
7,ДВФО,Владивосток,120
9,None,Владивосток,400


In [26]:
# шаг 3 - считаем функцию для этого ДФ
df_vladik['sales'].sum()

1000

Посчитать разницу максимального и минимального значения для города

In [28]:
df_vladik

,region,city,sales
2,ДВФО,Владивосток,290
4,ДВФО,Владивосток,100
5,ДВФО,Владивосток,50
6,ДВФО,Владивосток,40
7,ДВФО,Владивосток,120
9,None,Владивосток,400


In [30]:
max_value = df_vladik['sales'].max()
max_value

400

In [31]:
min_value = df_vladik['sales'].min()
min_value

40

In [32]:
max_value - min_value

360

In [34]:
def min_max(df):
    max_value = df['sales'].max()
    min_value = df['sales'].min()
    
    return max_value - min_value

In [ ]:
# Series - на входе значения столбца
# DataFrame - строки
# DataFrameGroupBy - на входе строки датафрейма с одинаковым городом

In [33]:
df.groupby('city')

In [35]:
df.groupby('city').apply(min_max)

city
Владивосток    360
Новосибирск     45
Томск           15
dtype: int64

### Метод agg
```python
.agg(lifetime = ('timestamp', min_max_diff), amount = ('timestamp', 'count' ))
```

In [5]:
def min_max_agg(sales_columns):
    """Считаем размах продаж для каждого города"""
    max_value = sales_columns.max()
    min_value = sales_columns.min()
    
    return max_value - min_value

In [6]:
df.groupby('city').agg({'sales': ['min', 'max', min_max_agg], 'region': 'count'})

sales                  region
              min  max min_max_agg  count
city                                     
Владивосток    40  400         360      5
Новосибирск    95  140          45      2
Томск         135  150          15      2

In [45]:
df.groupby('city').agg({'sales': ['min', 'max', lambda x: x.max() - x.min()], 'region': 'count'})

sales                 region
              min  max <lambda_0>  count
city                                    
Владивосток    40  400        360      5
Новосибирск    95  140         45      2
Томск         135  150         15      2

### Посчитать процент строки от ее региона

In [7]:
df.sort_values(['region', 'city'])

,region,city,sales
2,ДВФО,Владивосток,290
4,ДВФО,Владивосток,100
5,ДВФО,Владивосток,50
6,ДВФО,Владивосток,40
7,ДВФО,Владивосток,120
0,СФО,Новосибирск,140
3,СФО,Новосибирск,95
1,СФО,Томск,135
8,СФО,Томск,150
9,None,Владивосток,400


In [15]:
a = [1, 2, 3]
b = a.copy()  # вот это КОПИЯ

In [16]:
id(a) == id(b)

False

In [17]:
b.append(10)
b

[1, 2, 3, 10]

In [18]:
a

[1, 2, 3]

In [19]:
df_vladik = df.copy()
df_vladik = df_vladik[df_vladik['city'] == 'Владивосток']
df_vladik

,region,city,sales
2,ДВФО,Владивосток,290
4,ДВФО,Владивосток,100
5,ДВФО,Владивосток,50
6,ДВФО,Владивосток,40
7,ДВФО,Владивосток,120
9,None,Владивосток,400


In [20]:
total_sales = df_vladik['sales'].sum()
total_sales

1000

In [21]:
df_vladik['share'] = df_vladik['sales'] / total_sales
df_vladik

,region,city,sales,share
2,ДВФО,Владивосток,290,0.29
4,ДВФО,Владивосток,100,0.10
5,ДВФО,Владивосток,50,0.05
6,ДВФО,Владивосток,40,0.04
7,ДВФО,Владивосток,120,0.12
9,None,Владивосток,400,0.40


In [22]:
def share(df):
    total_sales = df['sales'].sum()
    df['share'] = df['sales'] / total_sales
    df['cumulative_sum'] = df['sales'].cumsum()
    
    return df

In [23]:
df.groupby('city').apply(share).sort_values(['city', 'region'])

ValueError: 'city' is both an index level and a column label, which is ambiguous.

In [68]:
df.groupby('city').apply(share).sort_values(['city', 'region']).drop(columns='cumulative_sum')

,region,city,sales,share
2,ДВФО,Владивосток,290,0.290000
4,ДВФО,Владивосток,100,0.100000
5,ДВФО,Владивосток,50,0.050000
6,ДВФО,Владивосток,40,0.040000
7,ДВФО,Владивосток,120,0.120000
9,None,Владивосток,400,0.400000
0,СФО,Новосибирск,140,0.595745
3,СФО,Новосибирск,95,0.404255
1,СФО,Томск,135,0.473684
8,СФО,Томск,150,0.526316


In [70]:
res = df.groupby('city').apply(share).sort_values(['city', 'region'])

del res['share']

res

,region,city,sales,cumulative_sum
2,ДВФО,Владивосток,290,290
4,ДВФО,Владивосток,100,390
5,ДВФО,Владивосток,50,440
6,ДВФО,Владивосток,40,480
7,ДВФО,Владивосток,120,600
9,None,Владивосток,400,1000
0,СФО,Новосибирск,140,140
3,СФО,Новосибирск,95,235
1,СФО,Томск,135,135
8,СФО,Томск,150,285


In [ ]:
df_1.merge(df_2, on='city')

# Тесты
Нужна функция, проверяющая корректность строки из трех столбцов:
- Идентификатор пользователя
- Возраст пользователя
- Профессия

Хотим по этой строке **определять возрастную категорию пользователя**.

Нужна функция, определяющая корректная строка на входе или нет
- Гораздо лучше разберетесь в задаче.
- Неплохая защита от ломающих изменений.

In [24]:
def is_correct(line):
    line_list = line.split(',')
    
    return len(line_list) == 3

In [25]:
line = '011,30,аналитик'  # то что на входе

In [26]:
is_correct(line)

True

In [ ]:
# < 18 - дети
# < 24 - учащиеся
# < 65 - трудящиеся
# > 65 - пенсионеры

In [81]:
'011,,'.split(',')

['011', '', '']

In [28]:
# юнит-тесты - напишем разные варианты входных данных
assert is_correct('011,30,аналитик')

assert not is_correct('011,,')  # это корректная строка?
assert not is_correct('011,30,')  # трудящиеся
assert not is_correct('011,56,')

# БД --> словарь профессий
# а если это заполняют пользователи?
assert not is_correct('011,56,оналитик данных-датасатанист')  # а как мы различаем вообще профессии?

assert not is_correct('')  # обязательно протестируйте ее на '', [], {}
# text[0]

AssertionError: 

Код функции

In [ ]:
def line_is_correct(line):
    values_list = line.split(',')
    
    if len(values_list) != 3:
        return False
    
    column_1, column_2, column_3 = values_list
    
    if not str.isnumeric(column_2):
        return False
    
    if not column_3.strip():
        return False
    
    return True